<a href="https://colab.research.google.com/github/NicCusinato/Webscraper/blob/main/Webscraper_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Webscraper 2.0 is used to show each dropdown for all the websites you gave it, can use it to manually compare what is available on each page**

# Setting up Seliunium Environment

In [2]:
import os
import time
!apt-get update
!apt install chromium-browser
!pip install google-colab-selenium
!pip install chromedriver-autoinstaller
import chromedriver_autoinstaller
import google_colab_selenium as gs



Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pac

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Run Environment

In [23]:
driver = gs.Chrome()

<IPython.core.display.Javascript object>

# Drop Down Extraction Function

In [36]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException, TimeoutException
import time

# Function to scrape dropdown (listbox) options from a page
def get_custom_dropdown_options(url):
    driver.get(url)
    time.sleep(2)  # Give the page some time to load

    # Find all button elements that act as listboxes
    dropdown_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[aria-haspopup="listbox"]')
    options_dict = {}

    # Loop through each dropdown button and extract options
    for index, button in enumerate(dropdown_buttons):
        try:
            # Scroll into view if necessary
            driver.execute_script("arguments[0].scrollIntoView(true);", button)
            time.sleep(0.5)  # Small delay to ensure smooth scrolling

            # Get the label of the dropdown, if available
            label_element = button.find_element(By.CLASS_NAME, 'swan-listbox-button-label')
            dropdown_label = label_element.text if label_element else f"Unnamed dropdown {index + 1}"

            # Click the button to expand the dropdown (listbox)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable(button)).click()
            time.sleep(1)  # Wait for the dropdown to expand

            # Find all the options within the listbox
            option_elements = driver.find_elements(By.CSS_SELECTOR, 'li[role="option"]')

            option_values = []
            for option in option_elements:
                data_key = option.get_attribute("data-key")  # Get the data-key attribute
                visible_text = option.find_element(By.CLASS_NAME, 'swan-listbox-option-text-container').text  # Get the visible text
                option_values.append({
                    "data_key": data_key,
                    "visible_text": visible_text
                })

            # Store the dropdown label and its associated options
            options_dict[dropdown_label] = option_values

            # Click somewhere outside the dropdown to close it (e.g., the body of the page)
            driver.find_element(By.TAG_NAME, 'body').click()
            time.sleep(0.5)  # Ensure the dropdown is fully closed before moving on
        except (ElementNotInteractableException, TimeoutException) as e:
            print(f"Could not interact with dropdown {index + 1}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

    return options_dict

# Comparision Function

In [28]:
# Function to compare dropdowns in both directions with exception handling
def compare_dropdowns(base_options, compare_options):
    comparison_results = {
        "missing_in_compare": {},  # Base page options missing in compare page
        "missing_in_base": {}      # Compare page options missing in base page
    }

    # Loop through base dropdown options and compare with compare_options
    for dropdown_id, base_vals in base_options.items():
        compare_vals = compare_options.get(dropdown_id, [])

        # Convert dictionaries to hashable frozensets based on their items
        base_set = set(frozenset(d.items()) for d in base_vals)
        compare_set = set(frozenset(d.items()) for d in compare_vals)

        # Handle missing dropdown keys using set comparison
        try:
            # Missing options in compare page
            missing_in_compare = base_set - compare_set
            if missing_in_compare:
                # Convert frozensets back to dictionaries
                missing_dicts = [dict(items) for items in missing_in_compare]
                comparison_results["missing_in_compare"][dropdown_id] = missing_dicts
        except KeyError:
            print(f"Dropdown ID '{dropdown_id}' not found in compare page options")

        try:
            # Missing options in base page
            missing_in_base = compare_set - base_set
            if missing_in_base:
                # Convert frozensets back to dictionaries
                missing_dicts = [dict(items) for items in missing_in_base]
                comparison_results["missing_in_base"][dropdown_id] = missing_dicts
        except KeyError:
            print(f"Dropdown ID '{dropdown_id}' not found in base page options")

    return comparison_results

# Scrape Pages

In [35]:
# Save dropdown options to a file
def save_to_file(filename, options_dict):
    with open(filename, 'w') as f:
        for dropdown_label, options in options_dict.items():
            f.write(f"Dropdown: {dropdown_label}\n")
            for option in options:
                f.write(f"Option: {option['visible_text']} (data-key: {option['data_key']})\n")
            f.write("\n")

# Define your base URL (the main product page you're comparing against)
base_url = "https://www.vistaprint.com/labels-stickers/custom-stickers/single-stickers"

# Define a queue of URLs to compare with the base page
urls_to_compare = [
    "https://www.vistaprint.com/labels-stickers/custom-stickers/die-cut-sticker-singles",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/holographic-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/face-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/party-stickers",
    "https://www.vistaprint.com/labels-stickers/custom-stickers/reusable-stickers"
    # Add more URLs here
]

# Get the dropdown options for the base page
base_dropdown_options = get_custom_dropdown_options(base_url)

# Save base page options to a file
save_to_file('base_page_options.txt', base_dropdown_options)

# Loop through the other pages and compare the dropdown options
for url in urls_to_compare:
    print(f"Comparing with {url}")
    compare_dropdown_options = get_custom_dropdown_options(url)

    # Save compare page options to a file
    filename = url.split("//")[1].replace("/", "_") + '_options.txt'
    save_to_file(filename, compare_dropdown_options)

    # Perform two-way comparison (this function is unchanged)
    comparison_results = compare_dropdowns(base_dropdown_options, compare_dropdown_options)

    if comparison_results["missing_in_compare"]:
        print(f"Options missing in {url} (compared to base page): {comparison_results['missing_in_compare']}")
    if comparison_results["missing_in_base"]:
        print(f"Options missing in base page (but present in {url}): {comparison_results['missing_in_base']}")
    if not comparison_results["missing_in_compare"] and not comparison_results["missing_in_base"]:
        print(f"All options match between base page and {url}.")

NameError: name 'NoSuchElementException' is not defined

# Close Environment

In [37]:
driver.quit()
